# Task 3a: Ingesting INRIX Metadata

* Review methods to store corridors (refer to the [CoA project](http://gitlab.utnmc.org/kap2357/inrix-corr) for guidance)
    - Run through the process of splitting off a separate INRIX corridors analysis from the CoA from [separate analysis](http://gitlab.utnmc.org/kap2357/inrix-corr/blob/master/doc/separate_analysis.md).
    - Run through the process of [staging scheme](https://github.com/TACC/ridi-dev/blob/ken_dev/docs/gen_db/staging_example.md) for two example studies.

* Ken can provide walkthrough of code and theory of operation (for ingesting new corridors)

* Look over [db_underlying.py](https://github.com/TACC/ridi-dev/blob/ken_dev/primer/inrix/db_underlying.py) and learn how to use if for this effort.


## Import Necessary Libraries

In [1]:
from importlib import reload

In [2]:
# Study id, title, and version
study_prov_name = "demo-study-1"
study_prov_title = "Demo Study 1"
study_src_ver = "0"
study_prov_desc = "Example test study to demonstrate ingestion process."

# Map version description
study_src_desc = "Segments for Demo Study collected from INRIX to demonstrate a Test Study (version 2402)."
map_src_ver_num = "2402"

data_src_id = "ixspd-demo1"

## Study and SQL Schema Data Definitions

In [ ]:
import dash # !pip install dash
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc # !pip install dash-bootstrap-components
import dash_leaflet as dl # !pip install dash-leaflet
from dash import dash_table # import dash_table
import ridi_tools
reload(ridi_tools)
from sqlalchemy import create_engine, text

# database connection (set these credentials to the ones of RIDI)
user_ = 'ridiadmin'
password_ = 'ridipass'
host_ = '127.0.0.1'
port_ = 5433
database_ = 'ridi_test'
engine = create_engine(f"postgresql://{user_}:{password_}@{host_}:{port_}/{database_}")

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.Div([
        html.H1("Initialize Ingestion", style={"textAlign": "center"}),
        html.H3("Fill out the information for the Ingestion Process.", style={"textAlign": "center", "font-weight": "normal"})
    ]),
    html.Div([
        dbc.Row([
            html.H4("Study Provider Information")
        ]),
        dbc.Row([
            dbc.Col([
                html.Label("Study ID: ", style={'margin-right': '10px'}), dcc.Input(id="study-id", placeholder="ID", value=study_prov_name if 'study_prov_name' in locals() else "")
            ]),
            dbc.Col([
                html.Label("Study Title: ", style={'margin-right': '10px'}), dcc.Input(id="study-title", placeholder="Title", value=study_prov_title if 'study_prov_title' in locals() else "")
            ]),
            dbc.Col([
                html.Label("Study Version: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-ver", placeholder="Source Version", value=study_src_ver if 'study_src_ver' in locals() else "")
            ])
        ], style={'margin-bottom': '10px'}),
        dbc.Row([
            html.Label("Study Description: "), dcc.Input(id="study-desc", placeholder="Description", value=study_prov_desc if 'study_prov_desc' in locals() else "")
        ], style={'margin-bottom': '10px', 'width': '100%'}),
        dbc.Row([
            html.H4("Study Source Information")
        ]),
        dbc.Row([
            html.Label("Source Description: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-desc", placeholder="Description", value=study_src_desc if 'study_src_desc' in locals() else "")
        ], style={'margin-bottom': '10px', 'width': '100%'}),
        dbc.Col([
            html.Label("Source Map Version: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-map-ver", placeholder="Map Version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")
        ], style={'margin-bottom': '10px', 'width': '100%'}),
        dbc.Row([
            html.H4("Aggregation")
        ]),
        dbc.Row([
            dcc.Checklist(
                id = "study-src-agg",
                options = [
                    {'label':'15 minutes', 'value':"15"},
                    {'label':'30 minutes', 'value':"30"},
                    {'label':'60 minutes', 'value':"60"}
                ],
                value = ['15'],
                inline = False
            )
        ], style={'margin-bottom': '10px'}),
        dbc.Row([
            html.H4("Data Source")
        ]),
        dbc.Row([
            dbc.Col([
                html.Label("Source ID: ", style={'margin-right': '10px'}), dcc.Input(id="src-id", placeholder="ID", value=data_src_id if 'data_src_id' in locals() else "ixspd")
            ]),
            dbc.Col([
                html.Label("Source Name: ", style={'margin-right': '10px'}), dcc.Input(id="src-name", placeholder="Name", value="INRIX IQ speed data")
            ])
        ], style={'margin-bottom': '10px'}),
        dbc.Row([
            html.Label("Source Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-desc", placeholder="Description", value="Data originating from INRIX IQ historic reports and live data streams.")
        ], style={'margin-bottom': '10px', 'width': "100%"}),
        dbc.Row([
            dbc.Col([
                html.Label("Source Corridor Version: ", style={'margin-right': '10px'}), dcc.Input(id="src-corr-ver", placeholder="ID", value=1)
            ]),
            dbc.Col([
                html.Label("Source Corridor Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-corr-desc", placeholder="Name", value="Corridor definition as represented in INRIX IQ report metadata.")
            ])
        ], style={'margin-bottom': '10px'}),
        dbc.Row([
            dbc.Col([
                html.Label("Source Segment Version: ", style={'margin-right': '10px'}), dcc.Input(id="src-seg-ver", placeholder="Description", value=1)
            ]),
            dbc.Col([
                html.Label("Source Segment Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-seg-desc", placeholder="Description", value="Segments as represented in INRIX IQ report metadata.")
            ])
        ])
    ],
    style={'margin-left': '50px'}),
    dbc.Row([
        dbc.Button("Initialize Ingestion", id="init-ingest-btn", color="secondary")
    ], style={'margin': "20px"}),
    dbc.Row([
        dcc.Textarea(id='terminal-output-init', readOnly=True, style={'height': '500px', 'width': '100%', 'background': 'black', 'color': '#32CD32', 'font':'consola'})
    ], style={'margin': '50px'})
])

@callback(
    Output('terminal-output-init', 'value'),
    Input('init-ingest-btn', 'n_clicks'),
    [
        State('terminal-output-init', 'value'),
        State('study-id', 'value'), State('study-title', 'value'), State('study-desc', 'value'),
        State('study-src-ver', 'value'), State('study-src-desc', 'value'), State('study-src-map-ver', 'value'), State('study-src-agg', 'value'),
        State('src-id', 'value'), State('src-name', 'value'), State('src-desc', 'value'),
        State('src-corr-ver', 'value'), State('src-corr-desc', 'value'), State('src-seg-ver', 'value'), State('src-seg-desc', 'value')
    ]
)
def ingest_init(n, current_output,
                study_prov_name, study_prov_title, study_prov_desc,
                study_src_ver, study_src_desc, map_src_ver_num, data_src_agg,
                src_id, src_name, src_desc, src_corr_ver, src_corr_desc, src_seg_ver, src_seg_desc):

    if not n:
        return current_output

    # if(study_prov_name and study_prov_title and study_prov_desc and
    #     study_src_ver and study_src_desc and map_src_ver_num and data_src_agg and
    #     src_id and src_name and src_desc and src_corr_ver and src_corr_desc and src_seg_ver and src_seg_desc):
    #     study_defs_sql = ridi_tools.study_schema_defs(
    #         study_provider_name=study_prov_name,
    #         study_provider_title=study_prov_title,
    #         study_provider_desc=study_prov_desc,
    #         study_source_ver=study_src_ver,
    #         study_source_study_desc=study_src_desc,
    #         map_source_version_num=map_src_ver_num,
    #         # data_source_part_id="ixspd",
    #         data_source_part_id=src_id,
    #         # data_source_part_name = "INRIX IQ test speed data (delete)",
    #         data_source_part_name=src_name,
    #         # data_source_part_desc = "Data originating from INRIX IQ historic reports and live data streams. (delete)",
    #         data_source_part_desc=src_desc,
    #         # data_source_part_corr_ver = 1,
    #         data_source_part_corr_ver=src_corr_ver,
    #         # data_source_part_corr_desc = "Corridor definition as represented in INRIX IQ report metadata. (delete)",
    #         data_source_part_corr_desc=src_corr_desc,
    #         # data_source_part_seg_ver = 1,
    #         data_source_part_seg_ver=src_seg_ver,
    #         # data_source_part_seg_desc = "Segments as represented in INRIX IQ report metadata. (delete)",
    #         data_source_part_seg_desc=src_seg_desc,
    #         data_source_part_data_aggregate=[[f"{agg}", 1, f"INRIX IQ speed data aggregated at {agg} minutes."] for agg in data_src_agg]
    #     )
    #     print(study_defs_sql)
    study_defs_sql = ridi_tools.study_schema_defs(
        study_provider_name=study_prov_name,
        study_provider_title=study_prov_title,
        study_provider_desc=study_prov_desc,
        study_source_ver=study_src_ver,
        study_source_study_desc=study_src_desc,
        map_source_version_num=map_src_ver_num,
        # data_source_part_id="ixspd",
        data_source_part_id=src_id,
        # data_source_part_name = "INRIX IQ test speed data (delete)",
        data_source_part_name=src_name,
        # data_source_part_desc = "Data originating from INRIX IQ historic reports and live data streams. (delete)",
        data_source_part_desc=src_desc,
        # data_source_part_corr_ver = 1,
        data_source_part_corr_ver=src_corr_ver,
        # data_source_part_corr_desc = "Corridor definition as represented in INRIX IQ report metadata. (delete)",
        data_source_part_corr_desc=src_corr_desc,
        # data_source_part_seg_ver = 1,
        data_source_part_seg_ver=src_seg_ver,
        # data_source_part_seg_desc = "Segments as represented in INRIX IQ report metadata. (delete)",
        data_source_part_seg_desc=src_seg_desc,
        data_source_part_data_aggregate=[[f"{agg}", 1, f"INRIX IQ speed data aggregated at {agg} minutes."] for agg in data_src_agg]
    )
    print(study_defs_sql)
    study_defs_sql_arr = study_defs_sql.strip().split("\n")
    # print(study_defs_sql_arr)

    # # Execute Ingestion in Database
    # updated_output = ""
    # for qry in study_defs_sql_arr:
    #     updated_output += qry + "\n"
    #     if("--" not in qry):
    #         try:
    #             conn = engine.connect()
    #             res = conn.execute(text(qry))
    #             conn.commit()
    #             conn.close()
    #             if(res.rowcount != -1):
    #                 updated_output += res.fetchall() + "\n"
    #             else:
    #                 updated_output += "No response.\n"
    #         except Exception as e:
    #             updated_output += "Skipping command.\n"

    updated_output = study_defs_sql + '\n'
    return updated_output # Clear input, keep output

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True)

-- Adding new Study
INSERT INTO provider (id, title, descr) VALUES ('demo-study-1', 'Demo Study 1', 'Example test study to demonstrate ingestion process.');
INSERT INTO source (provider, ver, title, study) VALUES ('demo-study-1', 0, 'Demo Study 1', 'Segments for Demo Study collected from INRIX to demonstrate a Test Study (version 2402).');
INSERT INTO source_part (id, source, part_type, def, descr) VALUES ('demo-study-1_corr-0', 'demo-study-1-0', 'L', 'corridor-1', 'Generic corridor definition');

-- Data definitions specific to INRIX; transcends map versions:
INSERT INTO provider (id, title, descr) VALUES ('ixspd-demo1', 'INRIX IQ speed data', 'Data originating from INRIX IQ historic reports and live data streams.');
INSERT INTO def (label, rev, descr) VALUES ('ixspd-demo1_corr', 1, 'Corridor definition as represented in INRIX IQ report metadata.');
INSERT INTO def (label, rev, descr) VALUES ('ixspd-demo1_seg', 1, 'Segments as represented in INRIX IQ report metadata.');
INSERT INTO de

In [ ]:
SELECT id, provider, title, ver, study FROM source; -- Check new study added

SELECT * FROM source_part; -- Check "_corr" and "_seg" rows added

## Ingest Study Segments (Metadata) into RIDI

In [4]:
# corridor ingestion
import sys
sys.path.append("/home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/primer/inrix")
import seg_ingest
# reload(seg_ingest)
import base64
import datetime
import io

import os
from pathlib import Path
import subprocess

# gui
import dash # !pip install dash
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc # !pip install dash-bootstrap-components
import dash_leaflet as dl # !pip install dash-leaflet
from dash import dash_table # import dash_table

# data frames
import pandas as pd

# necessary files
src_dir = "../downloads/inrix-nov-2024"
src_files = ["metadata.csv"]

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.Div([
        html.H1("Ingesting Segments (Metadata)", style={"textAlign": "center"}),
        html.H3("Fill out the information for the given segments.", style={"textAlign": "center", "font-weight": "normal"})
    ]),
    dbc.Container([
        dbc.Row([
            dbc.Col([
                html.Label("map version: ", style={"margin-right": "10px"}), dcc.Input(id="map-ver-seg", placeholder="map version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")
            ]),
            dbc.Col([
                html.Label("Metadata Source: ", style={"margin-right": "10px"}), dcc.Input(id="root-src-seg", placeholder="source", value=data_src_id if 'data_src_id' in locals() else "ixspd")
            ])
        ]),
        dbc.Row([
            dcc.Upload(
                id='upload-image',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                },
                accept = ".csv",
                multiple=True # don't allow multiple files
            )
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(
                    id="no-write-seg",
                    options=[{"label":"write", "value":False}, {"label":"don't write", "value":True}], # to write = false, to not write = true
                    value=True
                )
            ]),
            dbc.Col([
                dbc.Button("Ingest Segs", id="ingest-segs-btn", color="secondary")
            ])
        ]),
        dbc.Row([
            dcc.Textarea(id='terminal-output', readOnly=True, style={'height': '150px', 'width': '100%', 'background': 'black', 'color': '#32CD32', 'font':'consola'})
        ]),
        dbc.Row([
            dbc.Col([
                html.Div(id='output-image-upload')
            ])
        ])
    ])
])

def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if('csv' in filename):
            # Assume that the user uploaded a CSV file
            # print(filename)
            temp_meta_ptr = open("temp_metadata.csv", "w")
            temp_meta_ptr.write(decoded.decode('utf-8'))
            temp_meta_ptr.close()
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif('xls' in filename):
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            df.to_dict('records'),
            [{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])

@callback(
    Output('output-image-upload', 'children'),
    Input('upload-image', 'contents'),
    State('upload-image', 'filename'),
    State('upload-image', 'last_modified')
)
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)
        ]
        return children

@app.callback(
    Output('terminal-output', 'value'),
    Input('ingest-segs-btn', 'n_clicks'),
    [State('terminal-output', 'value'), State('map-ver-seg', 'value'), State('root-src-seg', 'value'), State('no-write-seg', 'value')] ## TODO: USE THE ROOT NAME TOOOOOOOOO
)
def segment_ingest(n_clicks, current_output, map_ver, src_id, no_write):
    if not n_clicks:
        return current_output

    output = ""
    if(map_ver):
        try:
            command = ""
            if(no_write):
                command = f"/home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/.venv/bin/python /home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/primer/inrix/seg_ingest.py -c ./temp_metadata.csv -r '{src_id}' -m {map_ver} -x"
            else:
                command = f"/home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/.venv/bin/python /home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/primer/inrix/seg_ingest.py -c ./temp_metadata.csv -r '{src_id}' -m {map_ver}"
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            output = (stdout.decode() + '\n' + stderr.decode()).strip()
            os.remove("temp_metadata.csv")
        except Exception as e:
            output = "Please select segment file (i.e. 'metadata.csv')."
    else:
        output = "map version is required."

    updated_output = output + '\n'
    return updated_output # Clear input, keep output

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True)


In [ ]:
SELECT DISTINCT source_part FROM ent; -- the new "<study_name>-<map_ver>_seg" will be added

SELECT * FROM ent WHERE source_part = 'ixspd-demo-2402_seg'; -- see actual segments added

## Ingest Study Corridors (Data) into RIDI

In [5]:
# corridor ingestion
import sys
sys.path.append("/home/programmedbean/files/nmc_projects/ridi/ridi-dev-jan-branch/primer/inrix")
import corr_ingest
reload(corr_ingest)
import ridi_tools
# reload(ridi_tools)
import db_underlying, inrix_model2, generic_model

# gui
import dash # !pip install dash
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc # !pip install dash-bootstrap-components
import dash_leaflet as dl # !pip install dash-leaflet
# import plotly.express as px

# data frames
import pandas as pd

# necessary files
src_dir = "../downloads/inrix-nov-2024"
src_files = ["corridor_segment.csv", "metadata.csv", "shapefile.csv"]

# get the corridor shapefiles
corr_geom, corr_roads = ridi_tools.get_corridor_shapefiles(src_dir, src_files)
num_of_corrs = len(corr_geom)
# print(f"{num_of_corrs} corridors found:")
# for n in range(len(corr_geom)):
#     print(f"corridor {n}:", corr_geom[n])

external_arrow = "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(
    [
        html.Div([
            html.H1("Naming Corridors", style={"textAlign": "center"}),
            html.H3("Select each corridor from the drop-down menu and give it a name.", style={"textAlign": "center", "font-weight": "normal"})
        ]),
        dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.Label("Study ID: ", style={"margin-right": "10px"}), dcc.Input(id="study-name", placeholder="study name", value=study_prov_name if 'study_prov_name' in locals() else "test-study-delete")
                ]),
                dbc.Col([
                    html.Label("Metadata Source: ", style={"margin-right": "10px"}), dcc.Input(id="root-src", placeholder="source", value=data_src_id if 'data_src_id' in locals() else "ixspd")
                ])
            ]),
            dbc.Row([
                dbc.Col(
                    [html.Label("map version: ", style={"margin-right": "10px"}), dcc.Input(id="map-version", type="number", placeholder="map version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")]
                ),
                dbc.Col(
                    [html.Label("aggregation: ", style={"margin-right": "10px"}), dcc.Input(id="map-aggregation", type="number", placeholder="aggregation", value=data_src_agg[0][0] if 'data_src_agg' in locals() else "")]
                )
            ])
        ]),
        dbc.Container([
            dbc.Row([
                dbc.Col(
                    dcc.Dropdown(
                        id="dropdown",
                        options=[{"label": "corridor "+str(n), "value": str(n)} for n in range(len(corr_geom))],
                        value="0",
                    )
                ),
                dbc.Col(dcc.Input(id="text", placeholder="type corridor name")),
                dbc.Col(dcc.Dropdown(
                    id="no-write",
                    options=[{"label":"write", "value":False}, {"label":"don't write", "value":True}], # to write = false, to not write = true
                    value=True
                )),
                dbc.Col(dbc.Button("Submit", id="name-corr-btn", color="secondary"))
            ]),
        ]),
        dl.Map(
            children = [
                dl.TileLayer()
            ],
            id = "dl-map",
            center = [30.398828, -97.684473], zoom=9.5, style={'height': '80vh'}
        ),
        dcc.Store(id="store", data={}),
        dcc.Store(id="count")
    ]
)

'''Change the geometry based on the corridor selected.'''
@app.callback(
    Output("dl-map", "children"),
    [Input("dropdown", "value")]
)
def change_corr(value):
    if(value is None):
        return [dl.TileLayer(), dl.Polyline(positions=corr_geom[0], color="red", children=[dl.Tooltip(corr_roads[0], permanent=True)])]
    curr_corr = int(value)
    return [dl.TileLayer(), dl.Polyline(positions=corr_geom[curr_corr], color="red", children=[dl.Tooltip(corr_roads[curr_corr], permanent=True)])]

'''Store the new corridor names.'''
@app.callback(
    Output("store", "data"),
    Output("text", "value"),
    Input("dropdown", "value"),
    Input("text", "value"),
    State("store", "data")
)
def update(val, text, data):
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]

    if input_id == "text":
        data[val] = text
        return data, dash.no_update
    else:
        return dash.no_update, data.get(val, "")

'''Confirm the corridor names and create the new corrs file'''
@app.callback(
    [Input("name-corr-btn", "n_clicks")],
    [State("store", "data"), State("map-version", "value"), State("map-aggregation", "value"), State("study-name", "value"), State("root-src", "value"), State("no-write", "value")]
)
def on_submit(n, corr_names, mapVer, agg, study_name, custom_root, no_write):
    if((len(corr_names) == num_of_corrs) and mapVer and agg and study_name):
        user_corr_names = list(corr_names.values())
        # print(user_corr_names)
        # print("mapVer:", mapVer)
        # print("agg:", agg)

        # # arguments of ingestion
        # custom_root = "ixspd"
        # study_name = "test-study-delete-0" # TODO: make these into fields

        if not custom_root:
            customRoot = inrix_model2.DEFAULT_ROOT

        newCorrs, segs = corr_ingest.readCorrsFile(src_dir, src_files, user_corr_names, mapVer, agg)
        if not newCorrs:
            raise Exception("No corridors contents found!")
        mapVer = newCorrs[0].map_id

        # Do everything:
        cmdSet = corr_ingest.ParseCmdLineRet(
            conn=db_underlying.DBConnBundle(),
            jsonFile=None,
            zipFile=None,
            corrFile=None,
            mapVer=mapVer,
            study=study_name,
            validateOnly=False,
            order=False,
            noBreak=False,
            customRoot=custom_root,
            noWrite=no_write
        )
        corr_ingest.linkCheckCorrs(cmdSet, newCorrs, segs, mapVer=mapVer, noWrite=no_write)

        if no_write:
            print("Nothing was written to the permanent database.")

    else:
        print("WARNING: not all corridors have been named.\nSelect each corridor from the drop-down and enter a name in the text box next to it.")
        if((mapVer == "") or (mapVer is None)):
            print("'map version' field is missing.")
        if((agg == "") or (agg is None)):
            print("'aggregation' field is missing.")
        print()

if __name__ == "__main__":
    app.run_server(mode="inline", debug=True)

In [ ]:
SELECT * FROM ent WHERE source_part = 'ixspd-demo-2402_corr';

In [11]:
# end job running extra dash-leaflet
!lsof -i :8050
# !kill -9 15711

COMMAND     PID           USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3.9 17977 programmedbean   67u  IPv4 622610      0t0  TCP localhost:8050 (LISTEN)


In [12]:
!kill -9 17977

: 

## TESTING ELEMENTS

In [ ]:
import dash
from dash import html, dcc, Input, Output, State

app = dash.Dash(__name__)

list_of_corrs = ["corr_1", "corr_2", "corr_3", "corr_4"]

app.layout = html.Div([
    dcc.Input(id='input-item', type='text', placeholder='Enter item'),
    html.Button('Add Item', id='add-button', n_clicks=0),
    html.Ul(id='item-list', children=[]),
])

@app.callback(
    Output('item-list', 'children'),
    Input('add-button', 'n_clicks'),
    State('input-item', 'value'),
    State('item-list', 'children')
)
def update_list(n_clicks, new_item, current_list):
    if n_clicks > 0 and new_item:
        current_list.append(html.Li(new_item))
    return current_list

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import pandas as pd

from collections import OrderedDict

from dash import Dash, Input, Output, State, dcc, html, callback
from dash.exceptions import PreventUpdate
from dash import dash_table


start_df = pd.DataFrame(
    OrderedDict([("Country", ["USA", "China", "France"])])
)
start_df["City"] = ""
start_df["Population"] = [31, 46, 11]

population_df = pd.DataFrame(
    OrderedDict(
        [
            ("Country", ["USA", "China", "China", "USA", "France"]),
            (
                "City",
                ["New-York", "Shanghai", "Beijing", "Los Angeles", "Paris",],
            ),
            ("Population", [19, 26, 20, 12, 11]),
        ]
    )
)

app = Dash(__name__)


app.layout = html.Div(
    [
        html.Div(
            html.H1("Country/City population"), style={"text-align": "center"}
        ),
        html.Hr(),
        dash_table.DataTable(
            id="table",
            columns=[
                {"id": "Country", "name": "Country",},
                {"id": "City", "name": "City", "presentation": "dropdown",},
                {"id": "Population", "name": "Population (Total [M])"},
            ],
            data=start_df.to_dict("records"),
            editable=True,
            dropdown_conditional=[
                {
                    "if": {
                        "column_id": "City",  # skip-id-check
                        "filter_query": '{Country} eq "China"',
                    },
                    "options": [
                        {"label": i, "value": i}
                        for i in population_df[
                            population_df.Country == "China"
                        ].City.values
                    ],
                },
                {
                    "if": {
                        "column_id": "City",  # skip-id-check
                        "filter_query": '{Country} eq "USA"',
                    },
                    "options": [
                        {"label": i, "value": i}
                        for i in population_df[
                            population_df.Country == "USA"
                        ].City.values
                    ],
                },
                {
                    "if": {
                        "column_id": "City",  # skip-id-check
                        "filter_query": '{Country} eq "France"',
                    },
                    "options": [
                        {"label": i, "value": i}
                        for i in population_df[
                            population_df.Country == "France"
                        ].City.values
                    ],
                },
            ],
            style_cell={
                "fontSize": "0.8rem",
                "whiteSpace": "normal",
                "padding": "3px",
                "textOverflow": "ellipsis",
                "textAlign": "center",
                "maxWidth": "300px",
            },
            style_header={
                "fontWeight": "500",
                "fontSize": "0.8rem",
                "cursor": "pointer",
            },
        ),
        html.Div(id="table_container"),
    ]
)

@callback(
    Output("table", "data"),
    Input("table", "data_timestamp"),
    State("table", "data"),
)
def update_table(timestamp, rows):
    print(timestamp)
    for row in rows:
        country = row["Country"]
        city = row["City"]
        if city == "" or city is None:
            print(country)
            population = start_df.set_index("Country").loc[country][
                "Population"
            ]
            print(population)
            row["Population"] = population
        elif city in population_df[["City"]].values:
            print(city)
            population = population_df.set_index("City").loc[city][
                "Population"
            ]
            print(population)
            row["Population"] = population

    return rows


if __name__ == "__main__":
    print(start_df)
    app.run(debug=True)

  Country City  Population
0     USA               31
1   China               46
2  France               11


None
USA
31
China
46
France
11
None
USA
31
China
46
France
11
None
USA
31
China
46
France
11
None
USA
31
China
46
France
11
1739901722693
USA
31
China
46
Paris
11
1739901728154
USA
31
Beijing
20
Paris
11
1739901731483
Los Angeles
12
Beijing
20
Paris
11
1739901733343
New-York
19
Beijing
20
Paris
11
1739901735128
Los Angeles
12
Beijing
20
Paris
11
1739901736538
New-York
19
Beijing
20
Paris
11
1739901738428
New-York
19
Beijing
20
Paris
11


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

user = 'ridiadmin'
password = 'ridipass'
host = '127.0.0.1'
port = 5433
database = 'ridi_test'

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
conn = engine.connect()

queries = ["CREATE TABLE delete_later();", "-- comment", "CREATE TABLE delete_after();"]


for q in queries:
    if("--" not in q):
        conn.execute(text(q))
conn.commit()
conn.close()

In [ ]:
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([

    dcc.Tabs(id="tabs", value='tab-1', children=[

        # Study and SQL Schema Data Definitions
        dcc.Tab(label='Study Definitions', value='study-defs', children=[
            html.Div([
                html.Div([
                    html.H1("Initialize Ingestion", style={"textAlign": "center"}),
                    html.H3("Fill out the information for the Ingestion Process.", style={"textAlign": "center", "font-weight": "normal"})
                ]),
                html.Div([
                    dbc.Row([
                        html.H4("Study Provider Information")
                    ]),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Study ID: ", style={'margin-right': '10px'}), dcc.Input(id="study-id", placeholder="ID", value=study_prov_name if 'study_prov_name' in locals() else "")
                        ]),
                        dbc.Col([
                            html.Label("Study Title: ", style={'margin-right': '10px'}), dcc.Input(id="study-title", placeholder="Title", value=study_prov_title if 'study_prov_title' in locals() else "")
                        ]),
                        dbc.Col([
                            html.Label("Study Version: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-ver", placeholder="Source Version", value=study_src_ver if 'study_src_ver' in locals() else "")
                        ])
                    ], style={'margin-bottom': '10px'}),
                    dbc.Row([
                        html.Label("Study Description: "), dcc.Input(id="study-desc", placeholder="Description", value=study_prov_desc if 'study_prov_desc' in locals() else "")
                    ], style={'margin-bottom': '10px', 'width': '100%'}),
                    dbc.Row([
                        html.H4("Study Source Information")
                    ]),
                    dbc.Row([
                        html.Label("Source Description: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-desc", placeholder="Description", value=study_src_desc if 'study_src_desc' in locals() else "")
                    ], style={'margin-bottom': '10px', 'width': '100%'}),
                    dbc.Col([
                        html.Label("Source Map Version: ", style={'margin-right': '10px'}), dcc.Input(id="study-src-map-ver", placeholder="Map Version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")
                    ], style={'margin-bottom': '10px', 'width': '100%'}),
                    dbc.Row([
                        html.H4("Aggregation")
                    ]),
                    dbc.Row([
                        dcc.Checklist(
                            id = "study-src-agg",
                            options = [
                                {'label':'15 minutes', 'value':"15"},
                                {'label':'30 minutes', 'value':"30"},
                                {'label':'60 minutes', 'value':"60"}
                            ],
                            value = ['15'],
                            inline = False
                        )
                    ], style={'margin-bottom': '10px'}),
                    dbc.Row([
                        html.H4("Data Source")
                    ]),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Source ID: ", style={'margin-right': '10px'}), dcc.Input(id="src-id", placeholder="ID", value=data_src_id if 'data_src_id' in locals() else "ixspd")
                        ]),
                        dbc.Col([
                            html.Label("Source Name: ", style={'margin-right': '10px'}), dcc.Input(id="src-name", placeholder="Name", value="INRIX IQ speed data")
                        ])
                    ], style={'margin-bottom': '10px'}),
                    dbc.Row([
                        html.Label("Source Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-desc", placeholder="Description", value="Data originating from INRIX IQ historic reports and live data streams.")
                    ], style={'margin-bottom': '10px', 'width': "100%"}),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Source Corridor Version: ", style={'margin-right': '10px'}), dcc.Input(id="src-corr-ver", placeholder="ID", value=1)
                        ]),
                        dbc.Col([
                            html.Label("Source Corridor Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-corr-desc", placeholder="Name", value="Corridor definition as represented in INRIX IQ report metadata.")
                        ])
                    ], style={'margin-bottom': '10px'}),
                    dbc.Row([
                        dbc.Col([
                            html.Label("Source Segment Version: ", style={'margin-right': '10px'}), dcc.Input(id="src-seg-ver", placeholder="Description", value=1)
                        ]),
                        dbc.Col([
                            html.Label("Source Segment Description: ", style={'margin-right': '10px'}), dcc.Input(id="src-seg-desc", placeholder="Description", value="Segments as represented in INRIX IQ report metadata.")
                        ])
                    ])
                ],
                style={'margin-left': '50px'}),
                dbc.Row([
                    dbc.Button("Initialize Ingestion", id="init-ingest-btn", color="secondary")
                ], style={'margin': "20px"}),
                dbc.Row([
                    dcc.Textarea(id='terminal-output-init', readOnly=True, style={'height': '500px', 'width': '100%', 'background': 'black', 'color': '#32CD32', 'font':'consola'})
                ], style={'margin': '50px'})
            ])
        ]),

        # Ingest Study Segments (Metadata) into RIDI
        dcc.Tab(label='Ingest Corridor Segments', value='ingest-seg', children=[
            html.Div([
                html.Div([
                    html.H1("Ingesting Segments (Metadata)", style={"textAlign": "center"}),
                    html.H3("Fill out the information for the given segments.", style={"textAlign": "center", "font-weight": "normal"})
                ]),
                dbc.Container([
                    dbc.Row([
                        dbc.Col([
                            html.Label("map version: ", style={"margin-right": "10px"}), dcc.Input(id="map-ver-seg", placeholder="map version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")
                        ]),
                        dbc.Col([
                            html.Label("Metadata Source: ", style={"margin-right": "10px"}), dcc.Input(id="root-src-seg", placeholder="source", value=data_src_id if 'data_src_id' in locals() else "ixspd")
                        ])
                    ]),
                    dbc.Row([
                        dcc.Upload(
                            id='upload-image',
                            children=html.Div([
                                'Drag and Drop or ',
                                html.A('Select Files')
                            ]),
                            style={
                                'width': '100%',
                                'height': '60px',
                                'lineHeight': '60px',
                                'borderWidth': '1px',
                                'borderStyle': 'dashed',
                                'borderRadius': '5px',
                                'textAlign': 'center',
                                'margin': '10px'
                            },
                            accept = ".csv",
                            multiple=True # don't allow multiple files
                        )
                    ]),
                    dbc.Row([
                        dbc.Col([
                            dcc.Dropdown(
                                id="no-write-seg",
                                options=[{"label":"write", "value":False}, {"label":"don't write", "value":True}], # to write = false, to not write = true
                                value=True
                            )
                        ]),
                        dbc.Col([
                            dbc.Button("Ingest Segs", id="ingest-segs-btn", color="secondary")
                        ])
                    ]),
                    dbc.Row([
                        dcc.Textarea(id='terminal-output', readOnly=True, style={'height': '150px', 'width': '100%', 'background': 'black', 'color': '#32CD32', 'font':'consola'})
                    ]),
                    dbc.Row([
                        dbc.Col([
                            html.Div(id='output-image-upload')
                        ])
                    ])
                ])
            ])
        ]),

        # 
        dcc.Tab(label='Ingest Corridors', value='ingest-corrs', children=[
            html.Div([
                html.H1("Naming Corridors", style={"textAlign": "center"}),
                html.H3("Select each corridor from the drop-down menu and give it a name.", style={"textAlign": "center", "font-weight": "normal"})
            ]),
            dbc.Container([
                dbc.Row([
                    dbc.Col([
                        html.Label("Study ID: ", style={"margin-right": "10px"}), dcc.Input(id="study-name", placeholder="study name", value=study_prov_name if 'study_prov_name' in locals() else "test-study-delete")
                    ]),
                    dbc.Col([
                        html.Label("Metadata Source: ", style={"margin-right": "10px"}), dcc.Input(id="root-src", placeholder="source", value=data_src_id if 'data_src_id' in locals() else "ixspd")
                    ])
                ]),
                dbc.Row([
                    dbc.Col(
                        [html.Label("map version: ", style={"margin-right": "10px"}), dcc.Input(id="map-version", type="number", placeholder="map version", value=map_src_ver_num if 'map_src_ver_num' in locals() else "")]
                    ),
                    dbc.Col(
                        [html.Label("aggregation: ", style={"margin-right": "10px"}), dcc.Input(id="map-aggregation", type="number", placeholder="aggregation", value=data_src_agg[0][0] if 'data_src_agg' in locals() else "")]
                    )
                ])
            ]),
            dbc.Container([
                dbc.Row([
                    dbc.Col(
                        dcc.Dropdown(
                            id="dropdown",
                            options=[{"label": "corridor "+str(n), "value": str(n)} for n in range(len(corr_geom))],
                            value="0",
                        )
                    ),
                    dbc.Col(dcc.Input(id="text", placeholder="type corridor name")),
                    dbc.Col(dcc.Dropdown(
                        id="no-write",
                        options=[{"label":"write", "value":False}, {"label":"don't write", "value":True}], # to write = false, to not write = true
                        value=True
                    )),
                    dbc.Col(dbc.Button("Submit", id="name-corr-btn", color="secondary"))
                ]),
            ]),
            dl.Map(
                children = [
                    dl.TileLayer()
                ],
                id = "dl-map",
                center = [30.398828, -97.684473], zoom=9.5, style={'height': '80vh'}
            ),
            dcc.Store(id="store", data={}),
            dcc.Store(id="count")
        ])
    ])
], style={'backgroundColor': 'white'})



if __name__ == '__main__':
    app.run_server(debug=True)